# Initialization

Test notebook for the C-MAPPS benchmark. Test using SVM. 

First we import the necessary packages and create the global variables.

In [1]:
#print(__doc__)

import math
import custom_scores
import CMAPSAuxFunctions
import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tunableModel import TunableModel

%matplotlib inline

# Create the model

In [2]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the results of running DE on the old model (250->50->1)
windowSize = 20
stride = 2
constRUL = 105

dataFolder = '../CMAPSSData'

#Create and compile the models
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SVM_RFB', svr_rbf, selected_features, dataFolder, 'scikit', window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)

# Load the data

In [3]:
#Selected from the results of running DE on the old model (250->50->1)
windowSize = 20
stride = 2
constRUL = 105
dataset = 1
tModel.changeDataset(dataset)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData(printTop=True)

Loading data for dataset 1 with window_size of 20, stride of 2 and constRUL of 105. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(9390, 280)
(9390, 1)
Testing data (X, y)
(100, 280)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.62199313 -0.07920059 -0.42905692 ..., -0.4         0.36        0.52472609]
 [-0.25773196 -0.16358253 -0.27612574 ..., -0.4         0.184       0.39798638]
 [-0.24398625 -0.41722181 -0.19031436 ..., -0.4         0.312       0.36955878]
 [-0.16838488  0.05008636 -0.54927782 ...,  0.          0.088       0.50370151]
 [-0.41580756 -0.0160375  -0.67544605 ..., -0.8         0.328       0.11637548]]
[[ 105.]
 [ 105.]
 [ 105.]
 [ 105.]
 [ 105.]]
Testing data (X, y)
[[-0.12714777 -0.19121638 -0.54715378 ..., -0.2         0.008       0.32691738]
 [ 0.01718213 -0.37132988 -0.03101105 ..., -0.6         0.008       0.05034054]
 [ 0.04467354 -0.10584752 -0.09005947 ...,  0.2         0.2         0.08646728]
 [-0.580756

# Train the regressor

In [12]:
#To change the model parameters the model has to be get first and then reinserted in the object
model = tModel.model
model.C = 1e4
model.epsilon = 0.2
model.gamma = 0.1
model.max_iter = 100
model.verbose = True
tModel.changeModel('ModelRUL_SVM_RFB', model, 'scikit')

#Train the model
tModel.trainModel()

[LibSVM]

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


# Evaluate the performance of the regressor

In [13]:
tModel.evaluateModel(["rhs"], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rectified RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

scores
Engine 1, RUL 86.23892120247264, Rectified RUL 86.0, Real RUL [ 112.]
Engine 2, RUL 93.01335059449957, Rectified RUL 93.0, Real RUL [ 98.]
Engine 3, RUL 54.93112150855232, Rectified RUL 54.0, Real RUL [ 69.]
Engine 4, RUL 75.96922872046741, Rectified RUL 75.0, Real RUL [ 82.]
Engine 5, RUL 92.24433007340869, Rectified RUL 92.0, Real RUL [ 91.]
Engine 6, RUL 83.45339622293615, Rectified RUL 83.0, Real RUL [ 93.]
Engine 7, RUL 89.20086770587588, Rectified RUL 89.0, Real RUL [ 91.]
Engine 8, RUL 80.79550328221241, Rectified RUL 80.0, Real RUL [ 95.]
Engine 9, RUL 89.48323386158063, Rectified RUL 89.0, Real RUL [ 111.]
Engine 10, RUL 79.60724151516987, Rectified RUL 79.0, Real RUL [ 96.]
Engine 11, RUL 76.7936560828835, Rectified RUL 76.0, Real RUL [ 97.]
Engine 12, RUL 86.17028177153807, Rectified RUL 86.0, Real RUL [ 124.]
Engine 13, RUL 80.8486555799432, Rectified RUL 80.0, Real RUL [ 95.]
Engine 14, RUL 85.12779377570644, Rectified RUL 85.0, Real RUL [ 107.]
Engine 15, RUL 84.63

In [ ]:
# #############################################################################
# Generate sample data
X = np.sort(5 * np.random.rand(40, 1), axis=0)
y = np.sin(X).ravel()

# #############################################################################
# Add noise to targets
y[::5] += 3 * (0.5 - np.random.rand(8))

# #############################################################################
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

# #############################################################################
# Look at the results
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()